In [8]:
import tensorflow as tf
import cv2
import numpy as np

# To deploy using keras saved model, requires more compute resources
# model = keras.models.load_model('saved_model_11')

# Deploying using tflite
interpreter = tf.lite.Interpreter('model_1.0.tflite')
interpreter.allocate_tensors()

input_height = 128
input_width = 128
n_channels = 1

class_no=3
labels = {0:'Normal', 1:'Bacterial Pneumonia', 2:'Viral Pneumonia'}


In [3]:
pip install customtkinter

     ------------------------------------ 296.1/296.1 kB 571.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
from tkinter import filedialog
import tkinter as tk
from PIL import ImageTk, Image
import customtkinter as ctk

import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.utils import load_img, img_to_array

class App(ctk.CTk):
    def __init__(self):
        super().__init__()

        self.title("Pneumonia Detection System")
        self.geometry(f"{500}x{740}")

  
       # Adding background image
        bg_image = Image.open("C:/Users/User/Desktop/project/Pneumonia-Detection/bg.jpeg")
        bg_image = bg_image.resize((1950, 1000))
        self.background_image = ImageTk.PhotoImage(bg_image)
        self.background_label = tk.Label(self, image=self.background_image)
        self.background_label.place(x=0, y=0, relwidth=1, relheight=1)

        self.head_frame = ctk.CTkFrame(master=self)
        self.head_frame.pack(pady=20, padx=60, fill="both", expand=True)
        self.main_frame = ctk.CTkFrame(master=self)
        self.main_frame.pack(pady=20, padx=60, fill="both", expand=True)
        self.head_label = ctk.CTkLabel(master=self.head_frame, text="Pneumonia Detection System",
                                       font=(ctk.CTkFont("Roboto"), 28))
        self.lis = []
        self.head_label.pack(pady=20, padx=10, anchor="nw", side="left")

        self.img_label = ctk.CTkButton(master=self.head_frame, text="Info", command=self.open_image_window,
                                       width=60, height=40)
        self.img_label.pack(pady=10, padx=10, anchor="nw", side="right")

        self.info_label = ctk.CTkLabel(master=self.main_frame,
                                       text="Pneumonia detection system, upload an X-ray image for Pneumonia detection.",
                                       wraplength=300, font=(ctk.CTkFont("Roboto"), 18))
        self.info_label.pack(pady=10, padx=10)

        self.upload_btn = ctk.CTkButton(master=self.main_frame, text="Upload Image", command=self.upload_image)
        self.upload_btn.pack(pady=0, padx=1)

        self.frame2 = ctk.CTkFrame(master=self.main_frame, fg_color="transparent", width=256, height=256)
        self.frame2.pack(pady=10, padx=1)

        img = Image.open("D:\\Downloads\\info.jpeg")
        img_resized = img.resize((int(256 / img.height * img.width), 256))
        img = ImageTk.PhotoImage(img_resized)
        self.img_label.configure(self.frame2,image=img, text="")
        self.img_label.image = img

        self.img_label = ctk.CTkLabel(master=self.frame2, text="", image=None)
        self.img_label.pack(pady=1, padx=10)

        self.predict_btn = ctk.CTkButton(master=self.main_frame, text="Pre-Process", command=self.preprocess)
        self.predict_btn.pack(pady=0, padx=1)

        self.result_frame = ctk.CTkFrame(master=self.main_frame, fg_color="transparent", width=200, height=100)
        self.result_frame.pack(pady=5, padx=5)

        self.save_btn = ctk.CTkButton(master=self.result_frame, text="Predict", command=self.predict_img)
        self.save_btn.pack(pady=0, padx=1)

        self.loader_label = ctk.CTkLabel(master=self.main_frame, width=100, height=100, text="")
        self.loader_label.pack(pady=3, padx=3)

        self.res1_label = ctk.CTkLabel(master=self.result_frame, text="")
        self.res1_label.pack(pady=5, padx=20)

        self.res2_label = ctk.CTkLabel(master=self.result_frame, text="")
        self.res2_label.pack(pady=5, padx=20)

    def upload_image(self):
        global filename
        filename = filedialog.askopenfilename(filetypes=(("Image files", "*.png;*.jpg;*.jpeg"), ("All files", "*.*")))

        self.res1_label.configure(text="")
        self.res2_label.configure(text="")
        self.img_label.configure(image=None)
        img = Image.open(filename)
        self.lis.append(filename)
        img_resized = img.resize((int(256 / img.height * img.width), 256))  # new width & height
        img = ImageTk.PhotoImage(img_resized)
        self.img_label.configure(self.frame2, image=img, text="")
        self.img_label.image = img

    def preprocess(self):
        file_path = self.lis[len(self.lis)-1]
        img = cv2.imread(file_path)
        chThr = [60,60]
        imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        imgBlur = cv2.GaussianBlur(imgGray,(0,0),1)
        ret, thresh1 = cv2.threshold(imgGray, 0, 150, cv2.THRESH_OTSU,cv2.THRESH_BINARY_INV)
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 30))
        imgCanny = cv2.Canny(imgBlur,chThr[0],chThr[1]) 
        image = Image.fromarray(imgCanny)
        image.thumbnail((400, 400))
        img_resized = image.resize((int(256 / image.height * image.width ), 256))  # new width & height
        img = ImageTk.PhotoImage(img_resized)
        self.img_label.configure(self.frame2, image=img, text="")
        self.img_label.image = img

    def open_image_window(self):
        im = Image.open("C:/Users/User/Desktop/project/Pneumonia-Detection/info.jpeg")
        im = im.resize((700, 700))
        im.show()

    def predict_img(self):
        image_path = self.lis[len(self.lis)-1]
        img = cv2.imread(image_path)
        img = cv2.resize(img, (input_height, input_width), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # To convert colormap from BGR to GRAY
        #Normalize the image - convert the each pixel value between 0 and 1
        img = img / 255
        img = np.reshape(img, (1, input_height, input_width, n_channels))
        img = np.float32(img)

        # predict result
        # prediction = model.predict(img)
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        interpreter.set_tensor(input_details[0]['index'], img)

        interpreter.invoke()
        prediction = interpreter.get_tensor(output_details[0]['index'])

        print(prediction)
        result = np.argmax(prediction)
        label = labels[result]
        confidence = prediction[0][result] * 100

        if label != 'Normal':
            self.res2_label.configure(text_color="RED", text="Result: " + label + " Confidence : "+ str(confidence), font=(ctk.CTkFont("Roboto"), 24))
        else:
            self.res2_label.configure(text_color="GREEN", text="Result: " + label + " Confidence : "+ str(confidence), font=(ctk.CTkFont("Roboto"), 24))


if __name__ == "__main__":
    app = App()
    app.mainloop()


TclError: image "pyimage2" doesn't exist